In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import re
import os
import glob
from string import punctuation
import csv
import textract
import pandas as pd
import numpy as np

In [37]:
documents = []
# PDF OKUMA
for pdf in glob.glob("Dokümanlar/*.pdf"):
    text = textract.process(pdf).decode('utf-8').replace("\n", " ").replace("\r", " ")
    #print(text)
    
    
    #PREPROCESS OF TEXT
    text = re.sub(r"İ", "i", text)
    text = re.sub(r"/", "",  text)
    text = re.sub(r"  ", " ", text)
    text = re.sub(r"\n", "", text)
    text = ''.join([c for c in text if c not in punctuation])
    text = text.lower()
    documents.append(text)
    

In [45]:
df = pd.DataFrame(documents, columns=["content"])
df.to_csv('datas.csv', index=False)

In [46]:
df = pd.read_csv("datas.csv")

In [47]:
df.content = df.content.apply(lambda x : str(x).lower())
df.content = df.content.apply(lambda x : re.sub( '\n', ' ', x ).strip())

In [48]:
from gensim import models
sentences = df.content.values.tolist()
# 
doc2vecSentence = [models.doc2vec.LabeledSentence(words=sentences[x], tags=["DOC_"+str(x)]) for x in range(0,len(sentences)) ]
model = models.Doc2Vec(dbow_words =1,
                        min_count = 50, 
                        epochs = 40 , 
                       )
model.build_vocab(doc2vecSentence)

/home/busra/PycharmProjects/Piton/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  after removing the cwd from sys.path.


In [49]:
X = list(model.docvecs.doctag_syn0)

/home/busra/PycharmProjects/Piton/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doctag_syn0` (Attribute will be removed in 4.0.0, use docvecs.vectors_docs instead).
  """Entry point for launching an IPython kernel.


In [50]:
from sklearn.cluster import KMeans
from sklearn import metrics
kmeans = KMeans(n_clusters=10)
kmeans.fit(X)
 
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
 
print ("Cluster id labels for inputted data")
print (labels)
print ("Centroids data")
print (centroids)
 
print ("Score (Opposite of the value of X on the K-means objective which is Sum of distances of samples to their closest cluster center):")
print (kmeans.score(X))
 
silhouette_score = metrics.silhouette_score(X, labels, metric='euclidean')
 
print ("Silhouette_score: ")
print (silhouette_score)


Cluster id labels for inputted data
[3 7 2 9 0 4 5 0 8 8 4 3 5 2 2 7 7 3 1 6 0 1 6 4 1 6 8 8 0 2]
Centroids data
[[-1.87233899e-03 -9.10037430e-04  1.19573759e-03 -1.57708180e-03
   7.25913560e-04 -1.65398221e-03  2.62224318e-03  2.34549632e-03
  -2.19104695e-03  1.98691132e-03  2.57252803e-04  1.68353199e-03
   1.12478474e-03  1.70478299e-03  4.16227063e-03  1.30374698e-03
  -2.04877404e-03 -3.61183396e-04  1.36438808e-03  5.07695580e-04
  -3.71280400e-03  6.58189674e-04  1.25854992e-03 -1.55357178e-03
   2.11161294e-03  1.22475757e-03  1.98540895e-03 -2.62802350e-05
   3.01645978e-03  5.46434392e-04 -2.13132422e-03  1.10243108e-03
   1.03326849e-03 -2.60547797e-03  2.15370240e-03 -3.29403940e-04
   4.75289024e-04 -1.74809613e-03  1.02648565e-03 -2.42429128e-03
  -2.48455742e-03 -4.78838687e-04  3.91675625e-04  2.69646762e-03
   4.24798665e-04  1.78822313e-03  5.88657625e-04 -2.24001415e-03
   1.32497813e-03 -1.84471239e-03 -2.30810807e-03 -3.06060917e-03
  -1.00843982e-03 -9.01423929

In [51]:
df["clust"] = labels

In [52]:
no_of_clusters=10

In [53]:
from sklearn.decomposition import NMF
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
nmf = NMF(n_components=no_of_clusters, 
                      random_state=100).fit_transform(scaler.fit_transform(X))
def find_max_index(li):
    # find_max val
    m = max(li)
    if li.count(m) >1:
        return -99
    else:
        return li.index(m)


In [54]:
import numpy as np
doc_topic_dist_unnormalized = np.matrix(nmf)
doc_topic_dist_unnormalized = [x.tolist()[0] for x in doc_topic_dist_unnormalized]
ref = list(zip(doc_topic_dist_unnormalized, list(df.content) ))
df2 = pd.DataFrame(ref)
df2.columns =["nmf", "Sentence"]
df2["cluster_no"] = df2["nmf"].apply(lambda x : find_max_index(x))

In [55]:
df2.groupby("cluster_no")["nmf"].count()

cluster_no
0    2
1    2
2    3
3    3
4    4
5    4
6    2
7    6
8    1
9    3
Name: nmf, dtype: int64

In [73]:

df["app2"] = df2.cluster_no
df["app1"] = df.clust
def remove_sw(x):
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize

    example_sent = x.lower()
    

        
    stop_words = set(stopwords.words('turkish'))
    stop_words = [x.lower() for x in stop_words]
    word_tokens = word_tokenize(example_sent)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = []

    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    return " ".join(filtered_sentence)

df["content2"] =  df["content"].apply(lambda x : remove_sw(x))

In [74]:
from math import ceil

nclust = no_of_clusters
ncols = 2
nrows = ceil(nclust/ncols)
import string



In [75]:
for i in range(nclust) :
    print("\n\n")
    # Removing stoipwords
    context = list(df[df.app1 == i].content2)
    from sklearn.feature_extraction.text import TfidfVectorizer
    m1 =TfidfVectorizer(max_features=20)
    m1.fit(context)
    print("Topic #{} : {}".format(i , " , ".join(m1.get_feature_names())))
    




Topic #0 : asliye , bir , dava , davacı , davalı , davanın , göre , hukuk , ilişkin , istanbul , karar , mahkemesi , olarak , olduğu , sayılı , tarafından , tenfiz , ticaret , ticari , şirketin



Topic #1 : asliye , aynı , bir , dava , davacı , davacının , davanın , esas , gereğince , hukuk , ilişkin , iptali , karar , maddesi , olarak , olduğu , sayılı , ticaret , çek , şartı



Topic #2 : bir , dava , davacı , davacının , davalı , diğer , hacze , hukuk , ihtiyati , istanbul , istinaf , itiraz , karar , mahkemesi , nedeniyle , olduğu , olduğunu , talep , yapılan , şirket



Topic #3 : adres , bedelli , ceza , dava , esas , göre , istinaf , karar , kararı , kayıt , olan , olarak , sayılı , tarafından , tarihli , tebligat , tebliğ , tl , yapılan , çek



Topic #4 : başlıklı , bir , dava , davacı , davacının , esas , hukuk , icra , istanbul , kanunun , karar , maddesi , menfi , olarak , sayılı , tespit , ticaret , ticari , çek , şartı



Topic #5 : bakırköy , bir , dava , davacı , da